In [ ]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array
from tensorflow.keras.applications.resnet50 import preprocess_input
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


In [ ]:
# Load data from the CSV file
data = pd.read_csv('/content/drive/MyDrive/Bacteria__Caracteristics.csv')

In [ ]:
# Load & preprocess the image
def load_and_preprocess_image(image_path):
    image = Image.open(image_path)
    image = image.resize((224, 224))
    image = img_to_array(image)
    image = preprocess_input(image)
    return image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Define the image directory path
image_dir = '/content/drive/MyDrive/Bacillus'

In [ ]:
# Load images and labels(classes)
images = []
labels = []
for class_name in os.listdir(image_dir):
    class_dir = os.path.join(image_dir, class_name)
    for image_filename in os.listdir(class_dir):
        image_path = os.path.join(class_dir, image_filename)
        images.append(load_and_preprocess_image(image_path))
        labels.append(class_name)

In [ ]:
# Encode the target labels (bacteria classes)
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(labels)

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Define the path to the directory containing the images for each class
class_dirs = ['/content/drive/MyDrive/Bacillus/Bacillus mycoides--aspect macroscopique', '/content/drive/MyDrive/Bacillus/Bacillus mycoides--aspect microscopique', '/content/drive/MyDrive/Bacillus/Bacillus subtilis--aspect macroscopique', '/content/drive/MyDrive/Bacillus/Bacillus subtilis--aspect microscopique']

In [ ]:
# Load images and bacterial characteristics
images = []
bacteria_labels = []

for i, class_dir in enumerate(class_dirs):
    class_name = label_encoder.classes_[i]
    class_images = [os.path.join(class_dir, filename) for filename in os.listdir(class_dir)]
    class_bacteria_labels = [class_name] * len(class_images)
    images.extend(class_images)
    bacteria_labels.extend(class_bacteria_labels)

In [ ]:
# Load and preprocess images
images = [load_and_preprocess_image(image_path) for image_path in images]

In [ ]:
   # Encode bacteria classes
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(data['Bacterie'])

In [ ]:

# Loading from csv
bacteria_features = data.drop(['Bacterie'], axis=1).values
bacteria_features = np.array([[1 if value == 'p' else 0 for value in row] for row in bacteria_features])

In [ ]:
print("Number of images:", len(images))   #check dossier des images
print("Number of bacteria features:", len(bacteria_features))   #File csv
print("Number of labels:", len(y))       #classes

Number of images: 52
Number of bacteria features: 52
Number of labels: 52


In [ ]:
# Split data into training and test sets
X_train_img, X_test_img, X_train_feat, X_test_feat, y_train, y_test = train_test_split(images, bacteria_features, y, test_size=0.2, random_state=42)

In [ ]:
# Load the pre-trained ResNet50 model
model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


In [ ]:
# Add a global average pooling layer and a dense layer for predictions
x = model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
bacteria_input = tf.keras.layers.Input(shape=(9,))  # Assuming you have 9 bacterial characteristics
combined = tf.keras.layers.concatenate([x, bacteria_input])
predictions = tf.keras.layers.Dense(len(label_encoder.classes_), activation='softmax')(combined)
custom_model = tf.keras.Model(inputs=[model.input, bacteria_input], outputs=predictions)


In [ ]:
# Compile the model
custom_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Combine image and feature data
X_train_combined = [(img, feat) for img, feat in zip(X_train_img, X_train_feat)]

# Create data generator for training
def generator():
    while True:
        for i in range(0, len(X_train_combined), batch_size):
            batch = X_train_combined[i:i + batch_size]
            X_batch_img = np.array([item[0] for item in batch])
            X_batch_feat = np.array([item[1] for item in batch])
            y_batch = y_train[i:i + batch_size]
            yield ([X_batch_img, X_batch_feat], y_batch)

batch_size = 32
train_steps_per_epoch = len(X_train_combined) // batch_size

train_generator = generator()


In [ ]:
# Combine image and feature data for validation
X_validation_combined = [(img, feat) for img, feat in zip(X_test_img, X_test_feat)]

# Create data generator for validation
def validation_generator():
    while True:
        for i in range(0, len(X_validation_combined), batch_size):
            batch = X_validation_combined[i:i + batch_size]
            X_batch_img = np.array([item[0] for item in batch])
            X_batch_feat = np.array([item[1] for item in batch])
            y_batch = y_test[i:i + batch_size]
            yield ([X_batch_img, X_batch_feat], y_batch)

validation_generator = validation_generator()


In [ ]:
custom_model.fit(train_generator, steps_per_epoch=train_steps_per_epoch, epochs=7)


# Save the model
custom_model.save('/content/drive/MyDrive/Custom_Model.h5')

# charger le modele
loaded_model = tf.keras.models.load_model('/content/drive/MyDrive/Custom_Model.h5')


Epoch 1/7
1/1 [==============================] - 43s 43s/step - loss: 4.5765 - accuracy: 0.0625
Epoch 2/7
1/1 [==============================] - 0s 171ms/step - loss: 5.6078 - accuracy: 0.0000e+00
Epoch 3/7
1/1 [==============================] - 0s 346ms/step - loss: 0.2932 - accuracy: 0.9375
Epoch 4/7
1/1 [==============================] - 0s 119ms/step - loss: 0.0624 - accuracy: 1.0000
Epoch 5/7
1/1 [==============================] - 0s 325ms/step - loss: 0.1080 - accuracy: 0.9688
Epoch 6/7
1/1 [==============================] - 0s 117ms/step - loss: 0.0075 - accuracy: 1.0000
Epoch 7/7
1/1 [==============================] - 0s 325ms/step - loss: 0.0190 - accuracy: 1.0000


In [ ]:
# Load and preprocess the test image
test_image_path = '/content/drive/MyDrive/Bacillus/Bacillus mycoides--aspect microscopique/image7.jpg'
test_image = load_and_preprocess_image(test_image_path)


In [ ]:
# Load characteristics from csv file
import os
import numpy as np

test_image_path_normalized = os.path.normpath(test_image_path)  # Normalisation
test_bacteria_row = data[data['Bacterie'] == test_image_path_normalized]

if not test_bacteria_row.empty:
    test_bacteria_row_index = test_bacteria_row.index
    data.loc[test_bacteria_row_index, 'Bacterie'] = test_image_path_normalized
    test_bacteria_features = data.loc[test_bacteria_row_index].drop(['Bacterie'], axis=1).values
    test_bacteria_features = np.where(test_bacteria_features == 'p', 1, 0)

else:
    print(f"No data found for image '{test_image_path_normalized}'.")


In [ ]:
test_image_filename = os.path.basename(test_image_path)
test_bacteria_row = data[data['Bacterie'].str.strip() == test_image_filename]
test_bacteria_row = data[data['Bacterie'].str.contains(test_image_filename, regex=False)]
test_bacteria_row = data[data['Bacterie'].astype(str) == str(test_image_filename)]

for value in data['Bacterie']:
    print(f"Comparing: '{value}' with '{test_image_filename}': {value == test_image_filename}")
test_bacteria_row = data[data['Bacterie'].str.split('/').str[-1] == test_image_filename]



Comparing: '/content/drive/MyDrive/Bacillus/Bacillus mycoides--aspect macroscopique/download01.jpg' with 'image7.jpg': False
Comparing: '/content/drive/MyDrive/Bacillus/Bacillus mycoides--aspect macroscopique/download1.jpg' with 'image7.jpg': False
Comparing: '/content/drive/MyDrive/Bacillus/Bacillus mycoides--aspect macroscopique/download2.jpg' with 'image7.jpg': False
Comparing: '/content/drive/MyDrive/Bacillus/Bacillus mycoides--aspect macroscopique/download3.jpg' with 'image7.jpg': False
Comparing: '/content/drive/MyDrive/Bacillus/Bacillus mycoides--aspect macroscopique/download4.jpg' with 'image7.jpg': False
Comparing: '/content/drive/MyDrive/Bacillus/Bacillus mycoides--aspect macroscopique/download5.jpg' with 'image7.jpg': False
Comparing: '/content/drive/MyDrive/Bacillus/Bacillus mycoides--aspect macroscopique/download6.jpg' with 'image7.jpg': False
Comparing: '/content/drive/MyDrive/Bacillus/Bacillus mycoides--aspect macroscopique/download7.jpg' with 'image7.jpg': False
Compari

In [ ]:
print("Provided Test Image Filename:", test_image_filename)
print("Matching Rows in DataFrame:")
print(bacteria_features)


Provided Test Image Filename: image7.jpg
Matching Rows in DataFrame:
[[1 1 0 0 1 1 1 1 1]
 [1 1 0 0 1 1 1 1 1]
 [1 1 0 0 1 1 1 1 1]
 [1 1 0 0 1 1 1 1 1]
 [1 1 0 0 1 1 1 1 1]
 [1 1 0 0 1 1 1 1 1]
 [1 1 0 0 1 1 1 1 1]
 [1 1 0 0 1 1 1 1 1]
 [1 1 0 0 1 1 1 1 1]
 [1 1 0 0 1 1 1 1 1]
 [1 1 0 0 1 1 1 1 1]
 [1 1 0 0 1 1 1 1 1]
 [1 1 0 0 1 1 1 1 1]
 [1 1 0 0 1 1 1 1 1]
 [1 1 0 0 1 1 1 1 1]
 [1 1 0 0 1 1 1 1 1]
 [1 1 0 0 1 1 1 1 1]
 [1 1 0 0 1 1 1 1 1]
 [1 1 0 0 1 1 1 1 1]
 [1 1 0 0 1 1 1 1 1]
 [1 1 0 0 1 1 1 1 1]
 [1 1 0 0 1 1 1 1 1]
 [1 1 1 1 1 1 0 1 0]
 [1 1 1 1 1 1 0 1 0]
 [1 1 1 1 1 1 0 1 0]
 [1 1 1 1 1 1 0 1 0]
 [1 1 1 1 1 1 0 1 0]
 [1 1 1 1 1 1 0 1 0]
 [1 1 1 1 1 1 0 1 0]
 [1 1 1 1 1 1 0 1 0]
 [1 1 1 1 1 1 0 1 0]
 [1 1 1 1 1 1 0 1 0]
 [1 1 1 1 1 1 0 1 0]
 [1 1 1 1 1 1 0 1 0]
 [1 1 1 1 1 1 0 1 0]
 [1 1 1 1 1 1 0 1 0]
 [1 1 1 1 1 1 0 1 0]
 [1 1 1 1 1 1 0 1 0]
 [1 1 1 1 1 1 0 1 0]
 [1 1 1 1 1 1 0 1 0]
 [1 1 1 1 1 1 0 1 0]
 [1 1 1 1 1 1 0 1 0]
 [1 1 1 1 1 1 0 1 0]
 [1 1 1 1 1 1 0 1 0]
 [1 1 1

In [ ]:
# predicted_probs = loaded_model.predict([np.array([test_image]), test_bacteria_features])
# predicted_class_index = np.argmax(predicted_probs, axis=-1)

#---------fonction de prediction
def predict_image(test_image, test_bacteria_features, Custom_Model, bacteria_name):
    predicted_probs = Custom_Model.predict([np.array([test_image]), test_bacteria_features])
    predicted_class_index = np.argmax(predicted_probs, axis=-1)
    predicted_class_name = label_encoder.classes_[predicted_class_index]

    return predicted_class_name



In [ ]:
# Calling function
bacteria_name = test_image_path.split('/')[5]  # Extract bacteria name from the image path 5eme champs
predicted_class_index = predict_image(test_image, test_bacteria_features, loaded_model, bacteria_name)

1/1 [==============================] - 1s 906ms/step


In [ ]:
#affichage
print(f"la bactérie prédite est: {bacteria_name}")

la bactérie prédite est: Bacillus mycoides--aspect microscopique
